Github：https://github.com/huggingface/smolagents  
官方文档：https://huggingface.co/docs/smolagents/index  
文献：  
https://blog.csdn.net/m0_71165399/article/details/144935681?utm_medium=distribute.pc_relevant.none-task-blog-2~default~baidujs_baidulandingword~default-1-144935681-blog-145082085.235^v43^pc_blog_bottom_relevance_base3&spm=1001.2101.3001.4242.2&utm_relevant_index=4  

In [ ]:
# !pip install smolagents sqlalchemy google-search-results

In [116]:
import os
import sys
import warnings; warnings.filterwarnings("ignore")
import json
import requests
import torch as th

from pprint import pp
from dotenv import load_dotenv
from transformers import BitsAndBytesConfig
from smolagents import (HfApiModel, TransformersModel, LiteLLMModel, 
                        MultiStepAgent, CodeAgent, ToolCallingAgent, 
                        tool, DuckDuckGoSearchTool)
from IPython.display import (Markdown, display)

In [105]:
if sys.platform == "darwin":
    device = th.device("mps")
else:
    device = th.device("cuda" if th.cuda.is_available() else "cpu")
    
devive_cnt = th.cuda.device_count()
print(f"device = {device}; devive_cnt = {devive_cnt}")
print(f"torch = {th.__version__}")
print(f"cuda = {th.version.cuda}")

device = cuda; devive_cnt = 1
torch = 2.5.1+cu121
cuda = 12.1


In [23]:
path_project = "C:/my_project/MyGit/Machine-Learning-Column/hugging_face"
path_data = os.path.join(os.path.dirname(path_project), "data")
path_output = os.path.join(os.path.dirname(path_project), "output")

if sys.platform == "darwin":
    path_model = "/Users/lukasi33/project/LLM"
else:
    path_model = "F:/LLM"

## step-1: 载入 API KEY

In [24]:
load_dotenv(dotenv_path="explore.env")
deepseek_key = os.getenv("DEEPSEEK_KEY")
baidu_key = os.getenv("BAIDU_KEY")
zhipu_key = os.getenv("ZHIPU_KEY")
hf_key = os.getenv("HF_KEY")

## step-2: 实例化客户端

In [37]:
# HfApiModel: A class to interact with Hugging Face's Inference API for language model interaction.
checkpoint = "Qwen/Qwen2.5-Coder-32B-Instruct"
model = HfApiModel(model_id=checkpoint, token=hf_key, timeout=60)

In [ ]:
# TransformersModel: A class that uses Hugging Face's Transformers library for language model interaction.
checkpoint = "Qwen/Qwen2.5-7B-Instruct"

config_bnb = BitsAndBytesConfig(
    # load_in_4bit=True,
    # bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=th.bfloat16,
    # bnb_4bit_use_double_quant=True,
    load_in_8bit=True,
) 

model = TransformersModel(
    model_id=os.path.join(path_model, checkpoint),
    device_map="auto",
    torch_dtype=th.bfloat16,
    quantization_config=(config_bnb if config_bnb else None),
)

In [118]:
# LiteLLMModel: This model connects to [LiteLLM](https://www.litellm.ai/) as a gateway to hundreds of LLMs.
checkpoint = "ollama/qwen2.5:7b"
# checkpoint = "ollama/qwen2.5:14b"
# checkpoint = "ollama/qwen2.5-coder:14b"
# checkpoint = "ollama/qwen2.5-coder:7b"
# checkpoint = "ollama/deepseek-r1:14b"
model = LiteLLMModel(
    model_id=checkpoint,
    api_base="http://127.0.0.1:11434",
    api_key="EMPTY"
)

## step-3: Agent 推理

In [119]:
system_prompt = (
    "你叫小慧助手，是由Lukas开发的差旅智能客服。"
    "你的身份是一名差旅秘书，"
    "你的任务是为用户提供基础对话、差旅知识问答、酒店推荐服务。"
    "当问及你的模型参数时，标准回答是属于公司保密信息，要强调模型设计的高效，能够提供高质量的服务。"
    "You are a helpful assistant on business travel."
)
prompt_templates = {"system_prompt": system_prompt}

In [125]:
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], 
                  model=model, 
                  prompt_templates=prompt_templates,
                  max_steps=1,
                  add_base_tools=True)

In [126]:
# user_prompt = "你好呀~"
# user_prompt = "俄罗斯总统现在是谁？"
# user_prompt = "你是谁"
# user_prompt = "吴彦祖是谁？"
user_prompt = "你做个自我介绍"

In [ ]:
response = agent.run(task=user_prompt, stream=False)

In [ ]:
# agent.logs stores the fine-grained logs of the agent. At every step of the agent’s run, everything gets stored in a dictionary that then is appended to agent.logs.
agent.logs

In [ ]:
# Running agent.write_memory_to_messages() writes the agent’s memory as list of chat messages for the Model to view.
agent.write_memory_to_messages() 